In [ ]:
import os
from google.colab import drive
from keras.preprocessing.image import ImageDataGenerator
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/DeepLearning/seq2seq')

Mounted at /content/drive


### Data : Tab-delimited Bilingual Sentence Pairs

In [ ]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from keras.optimizers import *
from keras.losses import sparse_categorical_crossentropy
import pandas as pd
# Path to translation file
path_to_data = 'hin.txt'

data=pd.read_csv("hin.txt",sep='\t',usecols=['Eng','Hin'])

In [ ]:
data

,Eng,Hin
0,Wow!,वाह!
1,Duck!,झुको!
2,Duck!,बतख़!
3,Help!,बचाओ!
4,Jump.,उछलो.
...,...,...
2944,"If you go to that supermarket, you can buy mos...",उस सूपरमार्केट में तुम लगभग कोई भी रोजाने में ...
2945,The passengers who were injured in the acciden...,जिन यात्रियों को दुर्घटना मे चोट आई थी उन्हे अ...
2946,"Democracy is the worst form of government, exc...","लोकतंत्र सरकार का सबसे घिनौना रूप है, अगर बाकी..."
2947,If my boy had not been killed in the traffic a...,अगर मेरा बेटा ट्रेफ़िक हादसे में नहीं मारा गया...


In [ ]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = string.punctuation + "!" + '.' + '।'
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [ ]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [ ]:
# Clean sentences
english_sentences = [clean_sentence(pair) for pair in data['Eng']]
hindi_sentences = [clean_sentence(pair) for pair in data['Hin']]

# Tokenize words
hin_text_tokenized, hin_text_tokenizer = tokenize(hindi_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length hindi sentence: {}'.format(len(max(hin_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))


# Check language length
hindi_vocab = len(hin_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("Hindi vocabulary is of {} unique words".format(hindi_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

Maximum length hindi sentence: 25
Maximum length english sentence: 22
Hindi vocabulary is of 2919 unique words
English vocabulary is of 2429 unique words


In [ ]:
max_hindi_len = int(len(max(hin_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

hin_pad_sentence = pad_sequences(hin_text_tokenized, max_hindi_len, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

# Reshape data
hin_pad_sentence = hin_pad_sentence.reshape(*hin_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [ ]:
# Encoder
input_sequence = Input(shape=(max_hindi_len,))
embedding = Embedding(input_dim=hindi_vocab, output_dim=128,)(input_sequence)
encoder = LSTM(64, return_sequences=False)(embedding)

# Decoder

input_sequence = Input(shape=(max_hindi_len,))
embedding = Embedding(input_dim=hindi_vocab, output_dim=128,)(input_sequence)
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(max_english_len)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)

enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])
enc_dec_model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 25, 128)           373632    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 22, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 22, 64)            33024     
_________________________________________________________________
time_distributed (TimeDistri (None, 22, 2429)          157885    
_________________________________________________________________
activation (Activation)      (None, 22, 2429)          0     

In [ ]:
model_results = enc_dec_model.fit(hin_pad_sentence, eng_pad_sentence, batch_size=30, epochs=100)


Epoch 1/100
99/99 [==============================] - 11s 74ms/step - loss: 3.8371 - accuracy: 0.7106
Epoch 2/100
99/99 [==============================] - 7s 74ms/step - loss: 2.1149 - accuracy: 0.7176
Epoch 3/100
99/99 [==============================] - 7s 74ms/step - loss: 1.9843 - accuracy: 0.7178
Epoch 4/100
99/99 [==============================] - 7s 74ms/step - loss: 1.9366 - accuracy: 0.7251
Epoch 5/100
99/99 [==============================] - 7s 74ms/step - loss: 1.9092 - accuracy: 0.7255
Epoch 6/100
99/99 [==============================] - 7s 73ms/step - loss: 1.8882 - accuracy: 0.7262
Epoch 7/100
99/99 [==============================] - 7s 74ms/step - loss: 1.8767 - accuracy: 0.7278
Epoch 8/100
99/99 [==============================] - 7s 75ms/step - loss: 1.8698 - accuracy: 0.7281
Epoch 9/100
99/99 [==============================] - 7s 74ms/step - loss: 1.8611 - accuracy: 0.7283
Epoch 10/100
99/99 [==============================] - 7s 74ms/step - loss: 1.8544 - accuracy: 0.728

In [ ]:
def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 6
print("The english sentence is: {}".format(english_sentences[index]))
print("The hindi sentence is: {}".format(hindi_sentences[index]))
print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(hin_pad_sentence[index:index+1])[0], eng_text_tokenizer))


The english sentence is: jump
The hindi sentence is: छलांग
The predicted sentence is :
jump <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty>
